In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import cv2
import numpy as np
import PIL
import scipy

Define paths

In [3]:
base_dir = 'D:/ML projects/Kitty mood/data/images' 

Data augmentation procedure

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 281 images belonging to 6 classes.


Found 67 images belonging to 6 classes.


Load the pre-trained MobileNetV2 model and fine-tune it

In [5]:
base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(6, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\Kovi\AppData\Local\Temp\ipykernel_40348\4292523440.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')


Train the model

In [6]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

Epoch 1/20


c:\Users\Kovi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.1628 - loss: 3.5889 - val_accuracy: 0.2985 - val_loss: 1.9705
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 621ms/step - accuracy: 0.1634 - loss: 2.9213 - val_accuracy: 0.2388 - val_loss: 1.9956
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 626ms/step - accuracy: 0.3053 - loss: 2.3972 - val_accuracy: 0.2537 - val_loss: 2.0864
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 628ms/step - accuracy: 0.2835 - loss: 2.6051 - val_accuracy: 0.2985 - val_loss: 1.7762
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 608ms/step - accuracy: 0.2729 - loss: 2.3314 - val_accuracy: 0.3433 - val_loss: 1.7389
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 604ms/step - accuracy: 0.3035 - loss: 2.1223 - val_accuracy: 0.2985 - val_loss: 1.7355
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 564ms/step - accuracy: 0.4116 - loss: 1.9946 - val_accuracy: 0.2985 - val_loss: 1.8908
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 562ms/step - accuracy: 0.3903 - loss: 1.8899 - val_accuracy: 0.3881 - val_loss: 1.683

Save the model

In [7]:
models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

model.save(os.path.join(models_dir, 'cat_mood_model.keras'))
print("Model saved successfully!")

Model saved successfully!


Load Model

In [10]:
model = tf.keras.models.load_model(os.path.join(models_dir, 'cat_mood_model.keras'))
print("Model loaded successfully!")

ValueError: Layer "dense" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 5, 5, 1280), dtype=float32, sparse=False, name=keras_tensor_1147>, <KerasTensor shape=(None, 5, 5, 1280), dtype=float32, sparse=False, name=keras_tensor_1148>]

Preprocess image

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

Predict mood

In [ ]:
def predict_mood(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    class_names = ['angry', 'calm', 'focused', 'groom', 'playful', 'sleepy']
    return class_names[np.argmax(prediction)]

Test the prediction function

In [ ]:
test_image_path = 'testimage/testimg.jpg' 
mood = predict_mood(test_image_path)
print(f'The cat is {mood}.')